In [3]:
!python -m venv venv

In [ ]:
!venv/Scripts/activate

In [ ]:
%pip install llama-index-embeddings-openai
%pip install llama-index-readers-file
%pip install python-dotenv

In [ ]:
%pip install -U qdrant_client
%pip install -U sentence-transformers

In [4]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["data/Almoravid+Rules+of+Play+-+LIVING+RULES+(1).pdf"]).load_data()

In [ ]:
#documents[0].text
for d in documents :
    print(d.text)

text = "\n".join([doc.text for doc in documents])
#TODO clear text

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

openai_key = os.getenv("OPENAI_API_KEY")

'abcd-fake'

In [23]:
print(len(text))

123521


In [27]:
from llama_index.core.node_parser import SentenceSplitter

# Initialize the SentenceSplitter
splitter = SentenceSplitter()

# Split the text into sentences
sentences = splitter.split_text(text)

print(len(sentences))


# Print the sentences
# for sentence in sentences:
#     print(sentence)

43


In [30]:
#sentence transformers chunkings
from sentence_transformers import SentenceTransformer

#model = SentenceTransformer('sentence-transformers/sentence-t5-large')
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

#TODO try different models https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

vectors = [model.encode(sentence) for sentence in sentences]

In [31]:
len(vectors)

43

In [ ]:
#TODO rest of semantic chunking https://docs.llamaindex.ai/en/stable/examples/node_parsers/semantic_chunking/

In [34]:
#Qdrant
from qdrant_client import models, QdrantClient
qdrant = QdrantClient(path="./qdrant_data")

In [33]:
len(vectors[0])

384

In [ ]:
#TODO get rid of deprecation warning
qdrant.recreate_collection(
    #TODO different names for different embeddings
    collection_name="demo_cosine",
    vectors_config=models.VectorParams(
        size=384, # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

C:\Users\kasta\AppData\Local\Temp\ipykernel_23632\1308980876.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [ ]:
[x for x in enumerate(sentences)]

In [ ]:
vectors[0]

In [43]:
qdrant.upsert(
    collection_name="demo_cosine",
    points=[
        models.PointStruct(
            id=idx, 
            vector=vectors[idx].tolist(),
            # place where dict with the vector source to be added or id/reference for the source
            payload={"source": doc},
        )
        for idx, doc in enumerate(sentences)
    ],
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [45]:
searchedText = """Battle
"""

In [50]:
hits = qdrant.search(
    collection_name="demo_cosine",
    query_vector=model.encode(searchedText).tolist(),
    limit=3,
)

for hit in hits:
    print(hit)
    print("score:", hit.score, "*****TXT******", hit.payload["source"][:80])

id=25 version=0 score=0.6054466467295004 payload={'source': 'NOTE: Battles last at least one Round.\n• This Round, \nthe Enemy gains a Pursuit advantage against the\nConceding side’s Hits (see below).\n• \n Place the Pursuit marker between the \nsides, pointing from Pursuing to Con-\nceding side.\nREPOSITION: Then, in each Round after the first, both sides \nreposition Lords as follows (in this order).\n• Rout. Remove \nRouted Lords from the Battle Array. They no\nlonger occupy any Array position nor participate further in the\nBattle until Ending the Battle (4.4.3).\n• Advance. \nSlide any Unrouted Lords in Reserve into any empty\nFront positions (one each).\n• Center.\n If a Front center position remains empty, first the At-\ntacker then the Defender must select and slide one of that side’s \nLords from either left or right front to fill its empty Front-cen-\nter position.\nSTRIKE: The Forces of Lords in Front Strike the Forces of the \nLord directly opposite them or—if Flanking—thos